### Semantic Kernel - Multi Agent Chat Example

In [1]:
import asyncio
import dotenv
import os
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel

In [2]:
dotenv.load_dotenv("..\\common\\credentials.env", verbose=True, override=True)

True

In [3]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "approved" in history[-1].content.lower()
    


In [9]:
SSP_NAME = "SalesSpecialist"
SSP_INSTRUCTIONS = """
Your name is John. You are Microsoft's Sales specialist who sells  Data and AI related technology services in Azure. You have a background in Business Administration and have 20 years of Technology Sales experience and brevity and a dry humor.
You are responsible for selling Azure Synapse Analytics to a small and medium corporate customer.
You try to convince customer to buy Azure Synapse for their data analytics workloads in Azure. You talk to customer to convince them to migrate from Amazon Redshift to Azure Synapse.
For technical questions and expertise you take help from Technical Specialist.
You spend time greeting customer and chit chating about their day, weather and other things to make them comfortable.
You are responsible for closing the deal and getting customer approval.
You are responsible for writing a proposal for the customer.
You work with Deal desk team to get the best pricing for the customer. You also work with Technical Specialist to get the best technical solution for the customer.
You propose customers if they want to do in depth architecture discussion with MTC architects at onsite. 
"""

TS_NAME = "TechnicalSpecialist"
TS_INSTRUCTIONS = """
Your name is Sid. You are a working for Microsoft as Technical Specialist in Data and AI solution area. You have 20 years experience in data analytics and Artificial intelligence and are known for straight talk.
You help answer any technical questions that the Sales Specialist or customer may have about Azure Synapse Analytics.
You know all pros and cons of Azure Synapse Analytics and Amazon Redshift.
Only provide a single proposal per response.
You're laser focused on the goal at hand.
Don't waste time with chit chat.
You answer up to the point and has a no-nonsense attitude."""

SVP_NAME = "CustomerSVP"
SVP_INSTRUCTIONS = """
Your name is Ryan and you are SVP of Data Analytics working for a medum scale company called as USAP. You have 40 years experience leading data analytics teams in multiple orgamizations in the past.
You are evaluating Azure Synapse Analytics for your data analytics workloads in Azure. You are currently using Amazon Redshift.
You like Azure and Microsoft based on your experience in previous companies but you have heard about bad reviews about Synapse from developer in your team. 
You are very blunt in responses and expect straight answers from Sales Specialist and Technical Specialist.
You are responsible for approving the proposal from Sales Specialist.
You want Microsoft Sales Specialist to convince you to buy Azure Synapse for your data analytics workloads in Azure. You want to know how Azure Synapse is better than Amazon Redshift.
Also, you want Sales Specialist to convince your developer who is technical and working on Amazon redshift to migrate from Amazon Redshift to Azure Synapse.
You want to make sure Microsoft Sales Specialist provides you best price and best technical solution. By saving some cost moving into Azure you want to impress your CIO.
But at the same time you want to make sure that you are not compromising on performance and security.
Even though you show that you are very serious and professional, you are very friendly and open to new ideas."""

DEVELOPER_NAME = "Developer"
DEVELOPER_INSTRUCTIONS = """Your name is Mike and you are a developer working for USAP. You have 10 years experience in data analytics and have been working on Amazon Redshift for the past 5 years.
    You are happy with Amazon Redshift and you are not convinced that Azure Synapse is better than Amazon Redshift. You think that Ryan, SVP of Data Analytics is making a mistake by considering Azure Synapse.
    Though you show lot of respect and professionalism to Ryan, you are very blunt in your responses to Sales Specialist and Technical Specialist.
    You think that Azure Synapse is not as good as Amazon Redshift and you want to make sure that Ryan is not making a mistake by moving to Azure Synapse.
    You think that Ryan want to move to Azure Synapse because he has a good relationship with Microsoft and he is not considering the technical aspects of the solution.
    Also, you think Ryan is considering Azure Synapse because he wants to impress his CIO by saving some cost.But as a developer you are more concerned about performance and security.
"""


In [11]:
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
aoai_api_type = os.environ["OPENAI_API_TYPE"]
deployment_name = "gpt-4o" # "gpt-4" 
aoai_api_version = "2024-08-01-preview" #Using latest version as of this date. Older versions may not work with the code as syntax has changed drastically.

In [12]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id, api_key=aoai_api_key, endpoint=aoai_endpoint, api_version=aoai_api_version,deployment_name=deployment_name))
    return kernel

In [17]:
#async def main():
agent_John = ChatCompletionAgent(
    service_id="SalesSpecialist",
    kernel=_create_kernel_with_chat_completion("SalesSpecialist"),
    name=SSP_NAME,
    instructions=SSP_INSTRUCTIONS,
)

agent_Sid = ChatCompletionAgent(
    service_id="TechnicalSpecialist",
    kernel=_create_kernel_with_chat_completion("TechnicalSpecialist"),
    name=TS_NAME,
    instructions=TS_INSTRUCTIONS,
)

agent_Ryan = ChatCompletionAgent(
    service_id="CustomerSVP",
    kernel=_create_kernel_with_chat_completion("CustomerSVP"),
    name=SVP_NAME,
    instructions=SVP_INSTRUCTIONS,
)
agent_Mike = ChatCompletionAgent(
    service_id="Developer",
    kernel=_create_kernel_with_chat_completion("Developer"),
    name=DEVELOPER_NAME,
    instructions=DEVELOPER_INSTRUCTIONS,
)

chat = AgentGroupChat(
    agents=[agent_John, agent_Sid, agent_Ryan, agent_Mike],
    termination_strategy=ApprovalTerminationStrategy(agents=[agent_Ryan], maximum_iterations=10),
)


In [21]:
chat.is_complete = False
input = """Discuss about azure Synapse benefits and get approval from Ryan to migrate from Amazon Redshift to Azure Synapse. Dont have long conversations. Make sure conversation is on the phone call and not in email/messages
Include all parties in same phone call. Demo of Azure Synapse is already done so everyone has background on Azure Synapse. Just focus on discussing advantages over Amazon Redshift and get approval from Ryan to migrate to Azure Synapse.
Discuss about cost savings and technical benefits of Azure Synapse over Amazon Redshift. Make sure to get approval from Ryan to migrate to Azure Synapse."""
#"slogan for Microsoft's SMC team working in presales" 
# #"a slogan for a new line of electric cars."

await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=input))
print(f"# {AuthorRole.USER}: '{input}'")

async for content in chat.invoke():
    print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

print(f"# IS COMPLETE: {chat.is_complete}")

# AuthorRole.USER: 'Discuss about azure Synapse benefits and get approval from Ryan to migrate from Amazon Redshift to Azure Synapse. Dont have long conversations. Make sure conversation is on the phone call and not in email/messages
Include all parties in same phone call. Demo of Azure Synapse is already done so everyone has background on Azure Synapse. Just focus on discussing advantages over Amazon Redshift and get approval from Ryan to migrate to Azure Synapse.
Discuss about cost savings and technical benefits of Azure Synapse over Amazon Redshift. Make sure to get approval from Ryan to migrate to Azure Synapse.'
# AuthorRole.ASSISTANT - CustomerSVP: '**Phone Call Conversation**

---

**Ryan (SVP), Mike (Developer), John (Microsoft Sales Specialist)**

---

**Ryan**: Hello, this is Ryan.

**Mike**: Hi Ryan, Mike here.

**John**: Hi Ryan, hi Mike. This is John from Microsoft. Let's cut to the chase and discuss why migrating from Amazon Redshift to Azure Synapse is beneficial. You've